In [1]:
# File dependencies
from tacc_stats.pickler.job_stats import Job
import cPickle as pickle
import argparse
import time as clock
from os import listdir
from IPython.display import clear_output

In [2]:
# Data manipulation dependencies
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

In [3]:
# Directory of all pickled jobs via comet
source_dir = '/oasis/projects/nsf/sys200/tcooper/xsede_stats/comet_pickles/'

# List of date directories in source_dir
dates_list = [ date for date in listdir(source_dir) ]

# Access and open pickled job files
**Process:**
    - Iterate through the non-empty date folders available in source_dir
    - A file is saved in valid_jobs if:
        * The pickled file is a Job object
        * The job ran for more than 6 cycles (1 hour)
        * The total number of jobs saved at the end of the previous date folder is less than 1000
            _This is purely to keep the computations manageable according to compute time requested_
    - Exceptions are skipped

In [4]:
n = float(sum([len(listdir(source_dir+date)) for date in dates_list]))
valid_jobs = []
t0 = clock.time()
count = 0

In [14]:
for date in dates_list:
    
    # do not open empty folders
    if len( listdir(source_dir+date) ) != 0 and len(valid_jobs) < 1000:
        size = len(listdir(source_dir+date))
        
        for job in listdir(source_dir+date):
            count += 1
            clear_output(wait=True)
            print("Processing file {} of {} files for {} \t ({}% of total)"
                  .format(count, size, date, np.round( count/n*100, 2)))
            
            # open job file if possible
            try:
                pickle_file = open( source_dir+date+'/'+job, 'rb')
                jobid = pickle.load(pickle_file)
                
                # only save jobs that ran longer than 1 hour
                if (len(jobid.times) > 5):
                    valid_jobs.append(jobid)
                    
                pickle_file.close()
                
            except:
                next 
            t2 = clock.time()
            print("total: {}s".format(np.round(t2-t0, 1)))
    else:
        next

In [6]:
# Check number of jobs accessed
len(valid_jobs)

73

# Format Columns & Clean Data

### Function: Given DataFrame, rename all columns with full label
<b>Notes:</b>
    - Certain column descriptions are repeated exactly in the available documentation. As a result, when these columns are relabelled according to their description, a column-specific identifier is appended in parentheses to keep it unique and prevent altering the meaning unintentionally.
    - Some of the intel categories are listed in the available documention as "-snb(hsw)-"; however, the code is actually tagged with "-hsw-". This is to note the respective categories are in fact included in this program, though they appear skipped.
    - At least one stat type was present in the data but does not appear to have a corresponding value in the available documentation, 'intel_rapl'. This has been interpretted to represent, "Running Average Power Limit" and is included in the proceeding analysis.

In [7]:
# Master list of all possible statistics collected
stat_types = ["amd64_pmc", "intel_hsw", "intel_hsw_ht", "intel_nhm",
              "intel_uncore", "intel_snb", "intel_rapl", "intel_hsw_cbo", "intel_hsw_pcu",
              "intel_hsw_imc", "intel_hsw_qpi", "intel_hsw_hau",
              "intel_hsw_r2pci", "ib", "ib_sw", "ib_ext", "llite",
              "lnet", "mdc", "mic", "osc", "block", "cpu", "mem", "net",
              "nfs", "numa", "proc", "ps", "sysv_shm", "tmpfs", "vfs", "vm"]

# Total number of categories
len(stat_types)

33

In [8]:
def formatTitles ( df ):
    return df.rename( columns={"index": "Job Name",
              "amd64_pmc": "AMD Opteron performance counters (per core)",
              "intel_hsw": "Intel Haswell Processor (HSW) (per core)",
              "intel_hsw_ht": "Intel Haswell Processor - Hyper-threaded (per logical core)",
              "intel_nhm": "Intel Nehalem Processor (NHM) (per core)",
              "intel_uncore": "Westmere Uncore (WTM) (per socket)",
              "intel_snb": "Intel Sandy Brige (SNB) or Ivy Bridge (IVB) Processor (per core)",
              "intel_rapl": "Running average power limit",
              "intel_hsw_cbo": "Caching Agent (CBo) for SNB (HSW) (per socket)",
              "intel_hsw_pcu": "Power Control Unit for SNB (HSW) (per socket)",
              "intel_hsw_imc": "Integrated Memory Controller for SNB (HSW) (per socket)",
              "intel_hsw_qpi": "QPI Link Layer for SNB (HSW) (per socket)",
              "intel_hsw_hau": "Home Agent Unit for SNB (HSW) (per socket)",
              "intel_hsw_r2pci": "Ring to PCIe Agent for SNB (HSW) (per socket)",
              "ib": "Infiniband usage (default)",
              "ib_sw": "InfiniBand usage (sw)",
              "ib_ext": "Infiniband usage (ext)",
              "llite": "Lustre filesystem usage (per mount)",
              "lnet": "Lustre network usage (lnet)",
              "mdc": "Lustre network usage (mdc)",
              "mic": "MIC scheduler account (per hardware thread)",
              "osc": "Lustre filesystem usage (osc)",
              "block": "Block device statistics (per device)",
              "cpu": "Scheduler accounting (per CPU)",
              "mem": "Memory usage (per socket)",
              "net": "Network device usage (per device)",
              "nfs": "NFS system usage",
              "numa": "NUMA statistics (per socket)",
              "proc": "Process specific data (MaxRSS, executable name etc.)",
              "ps": "Process statistics",
              "sysv_shm": "SysV shared memory segment usage",
              "tmpfs": "Ram-backed filesystem usage (per mount)",
              "vfs": "Dentry/file/inode cache usage",
              "vm": "Virtual memory statistics"
                            })

### Loops in loops in loops (Cleaning data)
**Notes:**
    - If a value is missing from the data, it will be replaced with '0' for the purpose of this project
    - If a type of statistic was not collected on the job, that column is dropped from the DataFrame
    - Two files are created during each iteration:
         1) A .csv of the descriptive statistics for that host,job pair
         2) A full .csv of the host,job data from the formatted DataFrame
    - Naming convention: Files are labelled as '{host}_{jobid}' to support random lookup
         * A job run on multiple host nodes is processed and saved with each individual host,job pair *

In [9]:
general = ["Overall Average", "Standard Deviation", "High", "Low"]

In [10]:
total = 0
n = len(valid_jobs)
t0 = clock.time()

for job in range( len(valid_jobs) ):
    clear_output(wait=True)

    # general job values
    jobid = valid_jobs[job]
    start = pd.to_datetime(round(jobid.start_time), unit='s').time()
    end = pd.to_datetime(round(jobid.end_time), unit='s').time()
    numCycles = len(jobid.times)
    total += 1
    type_avgs = {}
    times = []
    
    
    ##################################
    #  create DataFrame for schemas  #
    ##################################
    sch_tups = []
    
    for stat,schemas in jobid.schemas.iteritems():
        for schema in schemas.keys():
            sch_tups.append((stat,schema))
    
    # iterate through each host object job was run on
    for host_name, host in jobid.hosts.iteritems():
        try:
            devices = [ key for key in host.stats['intel_hsw'].keys() ]
            print("Processing hosts for job {} of {} \t ({}% of total)".format(job, n, np.round( job/n*100, 2)))
            
            ##################################
            #    convert timestamps to dt    #
            ##################################
            times.append(start)
            for time in host.times:
                times.append( pd.to_datetime(round(time), unit='s').time() )
            times.append(end)
            
            ##################################
            #   create DataFrame skeletons   #
            ##################################
            time_tups = [ (time,stat) for time in times for stat in general ]
            sch_idx = pd.MultiIndex.from_tuples(sch_tups, names=['Stat', 'Schema'])
            desc_idx = pd.MultiIndex.from_tuples(time_tups, names=['Stat', 'Schema'])
            
            schemas_df = pd.DataFrame( index=sch_idx, columns=times )
            intel_hsw_df = pd.DataFrame( index=times, columns=devices )
            desc_df = pd.DataFrame( index=desc_idx, columns=host.stats.keys() )
            job_df = pd.DataFrame( index=times, columns=host.stats.keys() )
            
            ##################################
            #   iterate through host.stats   #
            ##################################
            idx = 0
            
            for i in range( len(host.stats.keys())): 
                stat = host.stats.keys()[i]
                dev = host.stats[stat]
                
                for dev,cycles in dev.iteritems():                 
                    for j in range(len(cycles)):
                        time = times[j]
                        cycle = cycles[j]
                        
                        for k in range(len(cycle)):
                            if idx == len(sch_tups):
                                idx = 0
                            
                            loc = sch_tups[idx]
                            schemas_df.at[loc,time] = cycle[k]
                            idx += 1
                            
                        if stat == 'intel_hsw':
                            intel_hsw_df.loc[[time],[dev]] = cycle[8]           
                            
                        desc_df.loc[(time,"Overall Average"),[stat] ]    = np.mean(cycle )
                        desc_df.loc[(time,"Standard Deviation"),[stat] ] = np.std( cycle )
                        desc_df.loc[(time,"High"),[stat] ]               = np.max( cycle )
                        desc_df.loc[(time,"Low"),[stat] ]                = np.min( cycle )
                        
                        job_df.loc[[time],[stat]] = sum(cycle) 
                        
            #t2 = clock.time()
            #print("total: {}s".format(np.round(t2-t0, 1)))
            
            ###################################
            #    relable and format columns   #
            ###################################
            intel_hsw_df['Total'] = intel_hsw_df.sum(axis=1)
            desc_df = formatTitles( desc_df ).sort_index(axis=1)
            job_df = formatTitles( job_df ).T
            job_df.index.name = "Cycle"
            
            ###################################
            #   save DataFrames to csv files  #
            ###################################
            schemas_df.to_csv(path_or_buf="./jobs/by_schema/{}_{}.csv".format( host_name, jobid.id ))
            intel_hsw_df.to_csv(path_or_buf="./jobs/instructions_retired/{}_{}.csv".format( host_name, jobid.id ))
            desc_df.to_csv(path_or_buf="./jobs/descriptive_stats/{}_{}.csv".format( host_name, jobid.id ))
            job_df.to_csv(path_or_buf="./jobs/by_sum/{}_{}.csv".format( host_name, jobid.id ))
            
        except:
            next

Processing hosts for job 72 of 73 	 (0% of total)


In [12]:
# check that no job was missed
if total == len(valid_jobs):
    print "Success!"
else:
    print len(valid_jobs) - total, "jobs missing"

Success!
